In [4]:
from sys import path
path.append("..")
import numpy as np
from hypopt import GridSearch
from Models.Baselines import Baseline
from Utilities.dataformater import DataFormater
from FeatureEngineering_Selection.featureSelection import Selector
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from Models.prototypeEstimator import Prototype
from sklearn.ensemble import GradientBoostingClassifier,VotingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [5]:
def sklearnModelTuning(X_train,X_validation,X_test,y_train,y_validation,y_test):
    models = {
        "knn":KNeighborsClassifier,
        "log":LogisticRegression,
        "grad":GradientBoostingClassifier
        }
    paramSets = {
        "knn":{"n_neighbors":[5,15,25]},
        "log":{"C":[0.01,0.1,1,10,100]},
        "grad":{"max_depth":[2,3,4,5],"n_estimators":[100,150,200]}
    }
    searchers = [GridSearch(models[model](),param_grid=paramSets[model],parallelize=False) for model in models.keys()]
    for searcher in searchers:
       searcher.fit(X_train,y_train,X_validation,y_validation)
    return searchers

In [7]:
#Find best model params
formater = DataFormater()
X_train,X_validation,X_test,y_train,y_validation,y_test = formater.preProcessing(standardize=False,toNumpy=True)

for searchResult in sklearnModelTuning(X_train,X_validation,X_test,y_train,y_validation,y_test):
    # print(f"Score:{searchResult.best_score_} Best Params:{searchResult.best_params_}")
    print(f"Score:{searchResult.score(X_validation,y_validation)}")
    print(searchResult.best_params)

Score:0.7669902912621359
{'n_neighbors': 15}
Score:0.8058252427184466
{'C': 100}
Score:0.9029126213592233
{'max_depth': 4, 'n_estimators': 200}


(0.9029126213592233,
 0.7669902912621359,
 [0.5436893203883495,
  0.6601941747572816,
  0.8058252427184466,
  0.7766990291262136,
  0.9029126213592233,
  0.7669902912621359])

In [ ]:
def ensembleTuned(X_train,y_train,X_validation):
    '''Ensemble'''
    model = VotingClassifier([
        #("knn",KNeighborsClassifier(n_neighbors=15)),
        ("LogisticRegression",LogisticRegression(C=100)),
        ("gradientBoosting",GradientBoostingClassifier(max_depth=4,n_estimators=200))
        ],voting='soft')
    model.fit(X_train,y_train)
    return model.predict(X_validation)

yHat = ensembleTuned(X_train,y_train,X_validation)
print(accuracy_score(yHat,y_validation))

0.9029126213592233


In [ ]:
X_train,X_validation,X_test,y_train,y_validation,y_test = Selector().bestN(7,toNumpyFlag=True)
yHat = ensembleTuned(X_train,y_train,X_validation)
print(accuracy_score(yHat,y_validation))
#generated Features aren't useful with traditional models, but will be later...

0.8349514563106796
